In [1]:
import re
import pandas as pd

def extract_patent_data(file_path):
    # Read the OCR text file
    with open(file_path, "r", encoding="utf-8") as file:
        ocr_text = file.read()

    # More flexible BREVET pattern to handle OCR errors
    brevet_pattern = r"[\s/\\_\-]*B[R8][EÉÈÊË]V[EÉÈÊË]T"

    # Pattern to detect BREVET (D'... or DE ...) and capture details
    pattern = (
        rf"{brevet_pattern}\s+(?:D[’'\`]?|DE)\s*(?P<category>\w+)\s+DE\s+(?P<duration>\w+)\s+ANS[,\s]*\n?"
        r"(?P<content>.*?)(?=" + brevet_pattern + r"|\Z)"  # Capture until next "BREVET"
    )

    # Find page numbers from OCR tracking markers
    page_pattern = r"=== Extracted Text from page_(\d+)\.png ==="
    page_matches = list(re.finditer(page_pattern, ocr_text))
    page_positions = {match.start(): int(match.group(1)) for match in page_matches}

    # Split text by "BREVET" markers to find preceding content
    brevet_splits = re.split(brevet_pattern, ocr_text)

    patent_data = []
    last_page_number = None  # Track the last seen page number

    for i, match in enumerate(re.finditer(pattern, ocr_text, re.DOTALL)):
        full_text = match.group(0).strip()  # Full matched patent text
        category = match.group("category").strip()  # Patent type
        duration = match.group("duration")  # Duration
        content_snippet = match.group("content")  # Extract entire section between patents

        # Extract title (search in first 300 characters)
        title_match = re.search(r"Pour\s+(.+?)(?=\n|,|Au sieur|Aux sieurs|À madame)", content_snippet[:300])
        title = title_match.group(1).strip() if title_match else "Unknown Title"

        # Extract patentee
        patentee_match = re.search(r"(?:Au sieur|Aux sieurs|À madame) ([^.,\n]+)", content_snippet[:300])
        patentee = patentee_match.group(1).strip() if patentee_match else "Unknown Patentee"

        # Extract 100 characters BEFORE the current BREVET match position
        preceding_text_snippet = ocr_text[max(0, match.start() - 100): match.start()]

        # Extract the last non-empty line as a possible date
        possible_date_line = next((line.strip() for line in reversed(preceding_text_snippet.split("\n")) if line.strip()), "Unknown Date")

        #preceding_text = brevet_splits[i].strip().split("\n")[-5:]  # Last 5 lines before "BREVET"

        # Extract the last non-empty line (possible date)
        #possible_date_line = next((line.strip() for line in reversed(preceding_text) if line.strip()), "Unknown Date")

        # Try to extract a valid French-style date from the line
        date_match = re.search(r"\b(?:Le\s*)?(\d{1,2})\s+(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)\s+(\d{4})\b", possible_date_line, re.IGNORECASE)
        extracted_date = f"{date_match.group(1)} {date_match.group(2)} {date_match.group(3)}" if date_match else "Unrecognized Date"

        # Determine the closest page number before this patent
        patent_start = match.start()
        for pos in sorted(page_positions.keys(), reverse=True):
            if pos < patent_start:
                last_page_number = page_positions[pos]
                break

        # Calculate correct word and character count
        word_count = len(re.findall(r"\b\w+\b", content_snippet))  # Count all words
        char_count = len(content_snippet)

        # Store extracted data
        patent_data.append({
            "Patent Type": "BREVET",
            "Patent Category": category,
            "Duration (Years)": duration,
            "Patent Title": title,
            "Patentee": patentee,
            "Date Line": possible_date_line,  # Raw extracted line before BREVET
            "Extracted Date": extracted_date,  # Cleaned version if detected
            "Word Count": word_count,
            "Character Count": char_count,
            "Page Number": last_page_number if last_page_number else "Unknown Page",
            "Full Text": content_snippet.strip()
        })

    return pd.DataFrame(patent_data)




In [ ]:
# Example usage
file_path = "extracted_text_2.txt"
df = extract_patent_data(file_path)

In [12]:
df

,Patent Type,Patent Category,Duration (Years),Patent Title,Patentee,Date Line,Extracted Date,Word Count,Character Count,Page Number,Full Text
0,BREVET,INVENTION,CINQ,des procédés relatifs au laminage du fer et de,Jamar père et fils,4 septembre 1791.,4 septembre 1791,2521,14005,12,Pour des procédés relatifs au laminage du fer ...
1,BREVET,INVENTION,CINQ,un mécanisme propre à rendre régulière la fabri-,FureT - LapouLaYE,18 février 1796.,18 février 1796,231,1328,20,Pour un mécanisme propre à rendre régulière la...
2,BREVET,IMPORTATION,DIX,des bretelles et ceintures élastiques,John WaAuxer fils,17 GAOÎÛL 1791.,Unrecognized Date,507,2724,21,"Pour des bretelles et ceintures élastiques ,\n..."
3,BREVET,INVENTION,DIX,la construction de nouvelles grues,Baunrvrry-LiavA,12 décembre 1791.,12 décembre 1791,24,156,27,"Pour la construction de nouvelles grues , prop..."
4,BREVET,INVENTION,DIX,un peson ou romaine à cadran,HAmAzN père et fils,9 avril 1792.,9 avril 1792,38,214,27,"Pour un peson ou romaine à cadran,\nAux sieurs..."
...,...,...,...,...,...,...,...,...,...,...,...
74,BREVET,PERFECTIONNEMENT,CINQ,une machine à tondre les draps,LesLanc ParoissieN,"25 octobre 1803, et 27 juin 1806.",25 octobre 1803,699,3852,269,"Pour une machine à tondre les draps,\n\nAu sie..."
75,BREVET,INVENTION,DIX,des procédés relatifs à la teinture et à l’imp...,Dorrus,17 novembre 1798.,17 novembre 1798,1647,9341,272,Pour des procédés relatifs à la teinture et à ...
76,BREVET,INVENTION,CINQ,une cuisine économique,CoursAuT,ao décembre 1803.,Unrecognized Date,1036,5761,285,"Pour une cuisine économique ,\n\nAu sieur Cour..."
77,BREVET,INVENTION,QUINZE,un instrument nommé /le prompi Calculateur,John Firca,21 décembre 1793.,21 décembre 1793,584,3190,289,Pour un instrument nommé /le prompi Calculateu...


In [2]:
# Example usage
file_path = "extracted_text_12.txt"
df = extract_patent_data(file_path)

In [3]:
df

,Patent Type,Patent Category,Duration (Years),Patent Title,Patentee,Date Line,Extracted Date,Word Count,Character Count,Page Number,Full Text
0,BREVET,INVENTION,CINQ,une nouvelle pompe aspirante et foulante propre,Cxauvis et GurrLormN,31 décembre 1818.,31 décembre 1818,667,3705,12,Pour une nouvelle pompe aspirante et foulante ...
1,BREVET,PERFECTIONNEMENT,DIX,des perfectionnemens apportés aux procédés de,J,. 28 janvier 1814.,28 janvier 1814,2831,15802,15,Pour des perfectionnemens apportés aux procédé...
2,BREVET,INVENTION,CINQ,des moyens de dételer les chevaux qui s’empor-,Laurent Joawne,25 février 1819,25 février 1819,2746,15381,25,.\n\nPour des moyens de dételer les chevaux qu...
3,BREVET,INVENTION,CINQ,des perfectionnemens apportés aux presses hy-,GrNormsre,"21 ,uxllel 1819.",Unrecognized Date,659,3638,34,Pour des perfectionnemens apportés aux presses...
4,BREVET,IMPORTATION,QUINZE,une machine à rayer le papier,Decnanp,a1 juillet 1809.,Unrecognized Date,1915,10426,37,"Pour une machine à rayer le papier,\n\nAu sieu..."
...,...,...,...,...,...,...,...,...,...,...,...
67,BREVET,INVENTION,CINQ,un moyen de tremper les lames de rasoirs,Derronmis,27 septembre 1822.,27 septembre 1822,318,1809,310,(Déchu par Ordonnance royale du 16 soptembre 1...
68,BREVET,INVENTION,CINQ,des moyens de donner aux étoffes de laine,LacLoTTE,8 novembre 1822.,8 novembre 1822,216,1210,313,"(Déchu par Ordonnance royale du 4 mai 1825),\n..."
69,BREVET,INVENTION,CINQ,un fusil dit & capote,Alexis Mortau,9 février 1821.,9 février 1821,228,1201,315,"; fI\nPour un fusil dit & capote, samorçant av..."
70,BREVET,IMPORTATION,QUINZE,deux machines propres,Unknown Patentee,10 décembre 1813.,10 décembre 1813,2378,13167,343,"( Déchu par Ordonnance royale du 4 mai 1825), ..."


In [4]:
df.to_excel("ocr_extracted_patents_12_BW.xlsx", index=False)
